In [1]:
 pip install -U scikit-fuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 993 kB 9.3 MB/s 
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894089 sha256=cf65354b554ea969bf96ce974310e709b2a9e0431ddab993c03ed4926a88782b
  Stored in directory: /root/.cache/pip/wheels/d5/74/fc/38588a3d2e3f34f74588e6daa3aa5b0a322bd6f9420a707131
Successfully built scikit-fuzzy


In [2]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
import matplotlib.pyplot as plt

In [3]:
data = pd.read_excel('data_fuzzy.xlsx',sheet_name = ['Tingkat Kepentingan', 'MSGlow', 'Scarlet'])
df_tk = data['Tingkat Kepentingan']
df_msg = data['MSGlow']
df_sc = data['Scarlet']

In [4]:
def batas_bawah(data):
  col = data.columns
  data_batas_bawah = []
  for i in col:
    df = data[i]
    x = (1*df[0] + 1*df[1] + 2*df[2] + 3*df[3] + 4*df[4]) / sum(df)
    data_batas_bawah.append(x)
  return(data_batas_bawah)
  
def nilai_tengah(data):
  col = data.columns
  data_nilai_tengah = []
  for i in col:
    df = data[i]
    x = (1*df[0] + 2*df[1] + 3*df[2] + 4*df[3] + 5*df[4]) / sum(df)
    data_nilai_tengah.append(x)
  return(data_nilai_tengah)

def batas_atas(data):
  col = data.columns
  data_batas_atas = []
  for i in col:
    df = data[i]
    x = (2*df[0] + 3*df[1] + 4*df[2] + 5*df[3] + 5*df[4]) / sum(df)
    data_batas_atas.append(x)
  return(data_batas_atas)

def fuzzifikasi(data):
  Batas_Bawah = batas_bawah(data)
  Nilai_Tengah = nilai_tengah(data)
  Batas_Atas = batas_atas(data)
  df = pd.DataFrame(list(zip(Batas_Bawah, Nilai_Tengah, Batas_Atas)),columns =['batas bawah', 'nilai tengah', 'batas atas'])
  return(df)

In [5]:
fuzz_tk = fuzzifikasi(df_tk)
fuzz_msg = fuzzifikasi(df_msg)
fuzz_sc = fuzzifikasi(df_sc)

In [6]:
def defuzzifikasi(data):
  col = data.columns
  nrow = data.shape[0]
  defuzz = []
  for i in range(nrow):
    x = []
    for j in col:
      df = data[j]
      x.append(df[i])
    z = sum(x)/len(x)
    defuzz.append(z)
  return(defuzz)

In [7]:
fuzz_tk['defuzzifikasi'] = defuzzifikasi(fuzz_tk)
fuzz_msg['defuzzifikasi'] = defuzzifikasi(fuzz_msg)
fuzz_sc['defuzzifikasi'] = defuzzifikasi(fuzz_sc)

In [8]:
print(fuzz_tk)
print(fuzz_msg)
print(fuzz_sc)

   batas bawah  nilai tengah  batas atas  defuzzifikasi
0         3.33          4.32        4.76       4.136667
1         3.02          4.01        4.66       3.896667
2         3.03          4.03        4.75       3.936667
3         3.34          4.34        4.78       4.153333
4         2.84          3.84        4.54       3.740000
5         3.08          4.08        4.64       3.933333
   batas bawah  nilai tengah  batas atas  defuzzifikasi
0     3.133333      4.111111    4.711111       3.985185
1     2.888889      3.888889    4.622222       3.800000
2     3.022222      4.022222    4.755556       3.933333
3     3.177778      4.177778    4.711111       4.022222
4     2.555556      3.555556    4.422222       3.511111
5     2.488889      3.444444    4.266667       3.400000
   batas bawah  nilai tengah  batas atas  defuzzifikasi
0     3.036364      4.036364    4.745455       3.939394
1     3.072727      4.072727    4.781818       3.975758
2     3.145455      4.145455    4.781818       4

* Dari Tabel diketahui `tingat kepuasan merek MSGlow` yang memiliki nilai defuzzifikasi tertinggi adalah atribut `kualitas produk` dengan nilai sebesar 4,022

* Dari Tabel diketahui `tingat kepuasan merek Scarlet` yang memiliki nilai defuzzifikasi tertinggi adalah atribut `kemudahan dalam mencari produk` dengan nilai sebesar 4,024

In [11]:
import numpy as np

def divide_chunks(l, n):
  for i in range(0, len(l), n):
    yield l[i:i + n]

n = []

for i in range(fuzz_msg.shape[0]):
  X = (fuzz_tk.iloc[:,3][i]*fuzz_sc.iloc[:,3][i])
  for j in range(fuzz_sc.shape[0]):
    result = X - (fuzz_tk.iloc[:,3][j]*fuzz_msg.iloc[:,3][j])
    n.append(result)
 
teoper = pd.DataFrame(list(divide_chunks(n, 6)))
min = []
max = []

for i in range(teoper.shape[0]):
  min.append(np.min(teoper.iloc[i,:]))

for j in range(teoper.shape[0]):
  max.append(np.max(teoper.iloc[:,j]))

teoper['min'] = min
max.append(0)
teoper.loc[len(teoper.index)] = max
teoper.rename(index={6:'max'}, inplace=True)
teoper

,0,1,2,3,4,5,min
0,-0.189423,1.488626,0.811737,-0.409670,3.164404,2.922626,-0.409670
1,-0.993181,0.684869,0.007980,-1.213428,2.360646,2.118869,-1.213428
2,-0.643282,1.034768,0.357879,-0.863529,2.710545,2.468768,-0.863529
3,-0.023080,1.654970,0.978081,-0.243327,3.330747,3.088970,-0.243327
4,-1.910716,-0.232667,-0.909556,-2.130963,1.443111,1.201333,-2.130963
5,-1.967807,-0.289758,-0.966646,-2.188054,1.386020,1.144242,-2.188054
max,-0.023080,1.654970,0.978081,-0.243327,3.330747,3.088970,0.000000
